In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, confusion_matrix, PrecisionRecallDisplay, accuracy_score, matthews_corrcoef, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import math
from google.colab import files
uploaded = files.upload()

Saving training_df.tsv to training_df.tsv


# Functions for Feature Extraction

## Feature Extraction

In [ ]:
def sequence_composition(sequence:str) -> np.ndarray[np.float64]:
  sequence = sequence[:40]
  alphabet = "AQLSREKTNGMWDHFYCIPV"
  composition_matrix = np.zeros((20,40))
  composition_vector = np.zeros((20))
  for i in range(len(alphabet)):
    for j in range(len(sequence)):
      if alphabet[i] == sequence[j]:
        composition_matrix[i][j] += 1
        composition_vector[i] += composition_matrix[i][j]
  composition_vector = composition_vector/40
  return composition_vector

def hp_ai(sequence:str, window_size:int = 6 ) -> np.ndarray[np.float64]:
  kd_scale = {
    'R': -4.5,  # Arg
    'K': -3.9,  # Lys
    'N': -3.5,  # Asn
    'D': -3.5,  # Asp
    'Q': -3.5,  # Gln
    'E': -3.5,  # Glu
    'H': -3.2,  # His
    'P': -1.6,  # Pro
    'Y': -1.3,  # Tyr
    'W': -0.9,  # Trp
    'S': -0.8,  # Ser
    'T': -0.7,  # Thr
    'G': -0.4,  # Gly
    'A':  1.8,  # Ala
    'M':  1.9,  # Met
    'C':  2.5,  # Cys
    'F':  2.8,  # Phe
    'L':  3.8,  # Leu
    'V':  4.2,  # Val
    'I':  4.5   # Ile
    }

  a = 2.9 # relative volume of valine
  b = 3.9 # relative volume leucine/isoleucine

  sequence = sequence[:40]
  # padding to have as many scores as residues  (each window refers to the central res)
  d = int(window_size/2)
  sequence = "X"*d + sequence + "X"*d #padding
  hydrophobicities = np.array([])
  AIs = np.array([])  #aliphatic indexes
  for i in range(len(sequence)-(window_size)+1):
      counts_A = 0
      counts_V = 0
      counts_I = 0
      counts_L = 0
      hydrophobicity_score = 0
      window = sequence[i:i+window_size]
      for j in range(window_size):
        if window[j] != "X":
          hydrophobicity_score = (hydrophobicity_score + kd_scale[window[j]])
          if window[j] == 'A':
              counts_A += 1
          elif window[j] == 'V':
              counts_V += 1
          elif window[j] == 'I':
              counts_I += 1
          elif window[j] == 'L':
                counts_L += 1
      X_A = (counts_A / window_size)
      X_V = (counts_V / window_size)
      X_I = (counts_I / window_size)
      X_L = (counts_L / window_size)
      AI = X_A + a * X_V + b * (X_I + X_L) #Aliphatic Index Formula
      hydrophobicities = np.append(hydrophobicities, hydrophobicity_score / window_size)
      AIs = np.append(AIs, AI)
  H_AI = np.array([hydrophobicities.mean(), hydrophobicities.max(), np.argmax(hydrophobicities), AIs.mean(), AIs.max(), np.argmax(AIs)])
  return H_AI  #hydrophobicity and aliphatic index alltogether

def SSE(sequence,window_size):
  alpha_helix_scale = {
      "G": 0.570,  # Gly
      "P": 0.570,  # Pro
      "Y": 0.690,  # Tyr
      "C": 0.700,  # Cys
      "S": 0.770,  # Ser
      "T": 0.830,  # Thr
      "N": 0.670,  # Asn
      "R": 0.980,  # Arg
      "H": 1.000,  # His
      "D": 1.010,  # Asp
      "I": 1.080,  # Ile
      "W": 1.080,  # Trp
      "Q": 1.110,  # Gln
      "F": 1.130,  # Phe
      "K": 1.160,  # Lys
      "V": 1.060,  # Val
      "L": 1.210,  # Leu
      "A": 1.420,  # Ala
      "M": 1.450,  # Met
      "E": 1.510   # Glu
  }
  beta_sheet_scale = {
      "E": 0.370,  # Glu
      "D": 0.540,  # Asp
      "P": 0.550,  # Pro
      "G": 0.750,  # Gly
      "S": 0.750,  # Ser
      "K": 0.740,  # Lys
      "H": 0.870,  # His
      "N": 0.890,  # Asn
      "R": 0.930,  # Arg
      "A": 0.830,  # Ala
      "M": 1.050,  # Met
      "Q": 1.100,  # Gln
      "C": 1.190,  # Cys
      "T": 1.190,  # Thr
      "L": 1.300,  # Leu
      "F": 1.380,  # Phe
      "W": 1.370,  # Trp
      "Y": 1.470,  # Tyr
      "I": 1.600,  # Ile
      "V": 1.700   # Val
  }
  sequence = sequence[:40]
  # padding to have as many scores as residues (each window refers to the central res)
  d = int(window_size/2)
  sequence = "X"*d + sequence + "X"*d #padding
  alpha_helix = np.array([])
  beta_sheet = np.array([])
  for i in range(len(sequence)-(window_size)+1):
      alpha_score = 0
      beta_score = 0
      window = sequence[i:i+window_size]
      for j in range(window_size):
        w = 1 - abs(j - (window_size - 1)/2) / ((window_size - 1)/2)
        if window[j] != "X":
          alpha_score = alpha_score + w * alpha_helix_scale[window[j]]
          beta_score = beta_score + w * beta_sheet_scale[window[j]]
      alpha_helix = np.append(alpha_helix, alpha_score / window_size)
      beta_sheet = np.append(beta_sheet, beta_score / window_size)
  alpha_feature = np.array([alpha_helix.mean(), alpha_helix.max(), np.argmax(alpha_helix)])
  beta_feature = np.array([beta_sheet.mean(), beta_sheet.max(), np.argmax(beta_sheet)])
  return alpha_helix, alpha_feature, beta_sheet, beta_feature

def charge_seq(sequence, window_size):
  res_charges = {
        'K': 1,   # Lys
        'R': 1,   # Arg
        'H': 0.5, # His (partial positive)
        'D': -1,  # Asp
        'E': -1   # Glu
    }

  sequence = sequence[:20]
  # padding to have as many scores as residues  (each window refers to the central res)
  d = int(window_size/2)
  sequence = "X"*d + sequence + "X"*d #padding

  norm_charges = np.array([])

  for i in range(len(sequence)-(window_size)+1):
      charge = 0
      window = sequence[i:i+window_size]
      for j in range(window_size):
        if window[j] != "X" and window[j] in res_charges:
          charge = (charge + res_charges[window[j]])
      norm_charges = np.append(norm_charges, charge / window_size)
  charge_feature = np.array([norm_charges.max(), np.argmax(norm_charges), norm_charges.min(), np.argmin(norm_charges)])
  return norm_charges, charge_feature

def extract_features(seqs):
  features = np.empty([len(seqs), 36])

  for row, seq in enumerate(seqs):
    feature = np.array(())
    aa_comp = sequence_composition(seq)
    hyd_ali_index = hp_ai(seq)
    _, alpha, _, beta = SSE(seq, window_size=9)
    _, charges = charge_seq(seq, window_size=3)

    feature = np.concatenate((aa_comp, hyd_ali_index, alpha, beta, charges), axis = None)

    features[row] = feature
# Return individual feature vectors
  return features

## Cross validation cycle

In [ ]:
def cross_val(cycle_number:int):
  validation = []
  testing = []
  training = []
  for j in range(cycle_number):
    validation.append(j+1)
    testing.append(((j+1)%cycle_number)+1)
    training_j = []
    for i in range(cycle_number):
      if i != j and i != (j+1)%cycle_number:
        training_j.append(i+1)
    training.append(training_j)
    j +=1
  return training, validation, testing

# Cross Validation

### Feature extraction for the whole ds

In [ ]:
feature_names = ([f"Composition({residue})" for residue in "AQLSREKTNGMWDHFYCIPV"] + # 20 aa composition
 ["mean_hydrophobicity", "max_hydrophobicity", "most_hydrophobic_residue",
  "mean_aliphacity", "max_aliphacity", "most_aliphatic_residue", # 6 hydrophobicity and aliphacity
  "mean_alpha_helix", "max_alpha_helix", "most_likely_alpha_helix",
  "mean_beta_sheet", "max_beta_sheet", "most_likely_beta_sheet", # 6 accounting for SSE
  "max_charge", "most_charged", "min_charge", "less_charged" # 4 accounting for charge
  ])

# CREATING PANDAS SERIES FOR SEQUENCES AND CLASSES
df = pd.read_csv("training_df.tsv", sep = "\t")
training,validation,testing = [1,2,3],4,5
training_sequences = pd.Series([], dtype = object)
training_classes = pd.Series([], dtype = object)
validation_sequences = df[df['Label'] == validation]['Frag_90']
validation_classes = df[df['Label'] == validation]['Class']
testing_sequences = df[df['Label'] == testing]['Frag_90']
testing_classes = df[df['Label'] == testing]['Class']
for el in training:
        df_training = df[df['Label'] == el]
        training_sequences = pd.concat([training_sequences, df_training['Frag_90']])
        training_classes = pd.concat([training_classes, df_training['Class']])

#print(type(training_sequences), type(training_classes),
#      type(validation_sequences), type(validation_classes),
#      type(testing_sequences), type(testing_classes))

In [ ]:
# FEATURE EXTRACTION
training_features = extract_features(training_sequences)
validation_features = extract_features(validation_sequences)
testing_features = extract_features(testing_sequences)
print(training_features.shape)
print("")
print(validation_features.shape)
print("")
print(testing_features.shape)

In [ ]:
# FEATURE SCALING
def feature_scaler(features):
  scaler = StandardScaler()
  scaled_feature = scaler.fit_transform(features)
  return scaled_feature
training_features, validation_features, testing_features = feature_scaler(training_features), feature_scaler(validation_features), feature_scaler(testing_features)


### Baseline SVM (all features, manual grid over C and gamma on validation)

In [ ]:
# Baseline SVM (all features) — manual grid over C and gamma on validation

def svm_pipeline(C, gamma, kernel):
    return Pipeline([
        ("svm", SVC(kernel=kernel, C=C, gamma=gamma, random_state=42))
    ])

# grid parameters
kernels = ["linear", "rbf", "poly"]
C_grid = [0.1, 1, 10, 100]
gamma_grid = ["scale", 0.01, 0.1, 1.0]

best_score_base = -np.inf
best_params_base = None

for kernel in kernels:
  for C in C_grid:
      for gamma in gamma_grid:
          pipe = svm_pipeline(C, gamma, kernel)
          pipe = pipe.fit(training_features, training_classes)                 # fit on TRAIN
          pred_val = pipe.predict(validation_features)
          val_f1 = f1_score(validation_classes, pred_val)         # evaluate on VALIDATION
          if val_f1 > best_score_base:
              best_score_base = val_f1
              best_params_base = {"C": C, "gamma": gamma, "kernel": kernel}

print("Baseline SVM (all features) — best validation F1 score: "
      f"{best_score_base:.3f} with params {best_params_base}")

### Random forest gini importances

In [ ]:
# INITIALIZING RANDOM FOREST CLASSIFIER

rf = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    n_jobs=-1,
    class_weight={0: 9, 1: 1} # ADDRESSING CLASS UNBALANCE
)
rf.fit(training_features, training_classes)  # fit only on TRAIN
gini_imp = pd.Series(rf.feature_importances_, index = feature_names).sort_values(ascending=False)
gini_df = gini_imp.reset_index()
gini_df.columns = ["feature", "importance"]
print("Top 15 features by Gini importance:")
print(gini_df.head(15))

# Plot top 20
plt.figure()
plt.barh(gini_df["feature"].head(20)[::-1], gini_df["importance"].head(20)[::-1])
plt.xlabel("Gini importance")
plt.ylabel("Features")
plt.title("RandomForest Gini Importances (Top 20)")
plt.show()

### Top k features selection based on gini importance

In [ ]:
def f1_on_subset(C, gamma, kernel, subset_features):
    # subset by feature names
    feature_names_np = np.array(feature_names) # Convert to numpy array
    idx = [np.where(feature_names_np == f)[0][0] for f in subset_features]
    Xtr = training_features[:, idx]
    Xva = validation_features[:, idx]
    pipe = svm_pipeline(C, gamma, kernel)
    pipe = pipe.fit(Xtr, training_classes)     # train on TRAIN only
    pred = pipe.predict(Xva)     # predict on VAL only
    return f1_score(validation_classes, pred)  # f1 on VALIDATION

# We'll sweep k and, for each k, re-evaluate the best baseline SVM params on the reduced feature set
ks = list(range(2, min(30, training_features.shape[1]+1)))  # keep it small for speed/clarity
print(len(ks))
curve = []

for k in ks:
    subset = gini_df["feature"].head(k).tolist()
    f1_k = f1_on_subset(best_params_base["C"], best_params_base["gamma"], best_params_base["kernel"], subset)
    curve.append(f1_k)

best_k_idx = int(np.argmax(curve))
best_k = ks[best_k_idx]
print(f"Best k on validation (using baseline best params): k={best_k}, val_f1={curve[best_k_idx]:.3f}")

plt.figure()
plt.plot(ks, curve, marker="o")
plt.xlabel("k (top features by RF Gini)")
plt.ylabel("Validation F1 score (SVM)")
plt.title("F1 score vs. Number of Selected Features (Validation set)")
plt.grid(True)
plt.show()

### Retuning the SVM on the selected features

In [ ]:
# Use the best k from the validation curve
feature_names_np = np.array(feature_names)
best_subset = gini_df["feature"].head(best_k).tolist()
idx = [np.where(feature_names_np == f)[0][0] for f in best_subset]

Xtr_sel = training_features[:, idx]
Xva_sel = validation_features[:, idx]
Xte_sel = testing_features[:, idx]

# Manual grid search again but now restricted to the selected features
best_score_sel = -np.inf
best_params_sel = None

for kernel in kernels:
  for C in C_grid:
      for gamma in gamma_grid:
          pipe = svm_pipeline(C, gamma, kernel)
          pipe = pipe.fit(Xtr_sel, training_classes)      # train on TRAIN
          pred = pipe.predict(Xva_sel)
          val_f1 = f1_score(validation_classes, pred)  # validate on VAL
          if val_f1 > best_score_sel:
              best_score_sel = val_f1
              best_params_sel = {"C": C, "gamma": gamma, "kernel": kernel}

# Train final model on TRAIN+VAL with best params (optional) or just TRAIN; here we keep TRAIN only as per your outline
final_pipe = svm_pipeline(best_params_sel["C"], best_params_sel["gamma"], best_params_sel["kernel"])
final_pipe = final_pipe.fit(Xtr_sel, training_classes)
pred_test = final_pipe.predict(Xte_sel)
test_f1 = f1_score(testing_classes, pred_test)

print("Selected features (best k):", best_subset)
print("Best validation F1 score on selected features:", f"{best_score_sel:.3f}", "with", best_params_sel)
print("Test F1 score (selected features, tuned on val):", f"{test_f1:.3f}")

# For comparison: test accuracy with all features using baseline best params
baseline_pipe = svm_pipeline(best_params_base["C"], best_params_base["gamma"], best_params_base["kernel"])
baseline_pipe.fit(training_features, training_classes)
baseline_predtest = baseline_pipe.predict(testing_features)
test_f1_all = matthews_corrcoef(testing_classes, baseline_predtest)
print("Test F1 score (all features, baseline tuned on val):", f"{test_f1_all:.3f}")

In [ ]:
print(len(Xtr_sel[0]))

In [ ]:
# --- DATASET IMPORT ---
df = pd.read_csv("training_df.tsv", sep = "\t")

# --- CROSS-VALIDATION SETUP ---
cycles = 5
training, validation, testing = cross_val(cycles)

# --- INITIALIZE METRICS ---
confusions = [[] for _ in range(cycles)]
accuracies = np.array([])
precisions = np.array([])
recalls = np.array([])
f1s = np.array([])
mccs = np.array([])
ths = np.array([])

# --- CROSS-VALIDATION LOOP ---
      # During each cross-validation run,
      # each value in the grid is obtained with SVM models trained using the corresponding combination of C and 𝝲 and tested on the validation set
      # The best combination is selected as the one providingthe highest MCC on validation set

    # -------------------- TRAINING --------------------
      # FEATURE EXTRACTION
      # DEFINE MODEL WITH EVERY COMBINATION POSSIBLE OF THE HYPERPARAMETERS
    # -------------------- VALIDATION --------------------
      # EVALUATE BEST PARAMETERS AS THE MODEL WITH THE HIGHES MCC
      # KEEP THE MODEL WITH THE HIGHES MCC FOR TESTING
    # -------------------- TESTING --------------------
      # FEATURE EXTRACTION
      # PREDICTION WITH BEST MODEL
      # COMPUTE METRICS, STORE PARAMETERS
    # -------------------- METRICS --------------------
      # AFTER RUNNING EVERYTHING, HAVE A FINAL TABLE REPORTING THE PARAMETERS, THE METRICS FOR EACH RUN

for i in range(cycles):
    print(f"\n{'='*30}\nROUND {i+1}\n{'='*30}")

    mySVC = svm.SVC(C=1.0, kernel='rbf', gamma = 'scale')

    # -------------------- TRAINING --------------------

    # -------------- FEATURE EXTRACTION ----------------
    training_seqs = pd.Series([], dtype=object)
    training_classes = pd.Series([], dtype = object)
    for el in training[i]:
        df_training = df[df['Label'] == el]
        training_seqs = pd.concat([training_seqs, df_training['Frag_90']])
        training_classes = pd.concat([training_classes, df_training['Class']])
    #training_classes.to_numpy()

    #training_seqs = training_seqs.reset_index(drop=True)  # ensure sequential index

    training_feats = extract_features(training_seqs)
    print(f"features_{i+1}")
    print(training_feats)
    print("")

    # -------------------- MODEL FITTING -----------------

    mySVC.fit(training_feats, training_classes)

    # -------------------- VALIDATION --------------------
    sequences_val = df[df['Label'] == validation[i]]['Frag_90']
    validation_classes = df[df['Label'] == validation[i]]['Class']
    validation_classes.to_numpy()

    print(f"Validation Set Round {i+1}")
    print(sequences_val.head(2))
    print(sequences_val.tail(2))

    validation_scores = np.array([])
    for seq in sequences_val:
        score = compute_score(seq, 15, pswm)
        validation_scores = np.append(validation_scores, score)

    print(f"Validation sequences processed: {len(validation_scores)}\n")

    # -------------------- THRESHOLD SELECTION --------------------
    precision_vals, recall_vals, thresholds = precision_recall_curve(validation_classes, validation_scores)
    fscore = (2 * precision_vals * recall_vals) / (precision_vals + recall_vals + 1e-15)
    index = np.argmax(fscore)
    optimal_threshold = thresholds[index]

    print(f"Optimal threshold for round {i+1}: {optimal_threshold}\n")

    # -------------------- TESTING --------------------
    testing_sequences = df[df['Label'] == testing[i]]['Frag_90']
    testing_classes = df[df['Label'] == testing[i]]['Class']

    testing_scores = np.array([compute_score(seq, 15, pswm) for seq in testing_sequences])

    predictions = np.where(testing_scores >= optimal_threshold, 1, 0)

    print(f"Testing Set Round {i+1}")
    print(testing_sequences.head(2))
    print(testing_sequences.tail(2))
    print(f"Testing samples: {len(testing_sequences)}\n")

# --- SUMMARY ---
print("\n===== CROSS-VALIDATION SUMMARY =====")
print(f"Accuracies: {accuracies}")
print(f"Precisions: {precisions}")
print(f"Recalls: {recalls}")
print(f"F1-scores: {f1s}")
print(f"MCCs: {mccs}")
print(f"Thresholds: {ths}")
print(f"Confusion matrices: {confusions}")


(8021, 5)
